In [5]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from scipy.optimize import curve_fit
import glob
import os


# === CONFIGURATION ===
image_folder = "/content/drive/MyDrive/DATA/AMP_AMP/petri0/jpg"
image_paths = sorted(glob.glob(os.path.join(image_folder, "*.jpg")))[15:48]
gradient_direction = 'right'  # Direction of antibiotic gradient
max_conc = 29.2               # Maximum concentration on plate
radius_reduction = 30         # Radius for excluding colony edge
start_time_hr = 7.5
image_interval_hr = 0.5
start_index = int(start_time_hr / image_interval_hr)

image_paths = sorted(image_paths)[start_index:start_index + 48]
times = np.arange(len(image_paths)) * image_interval_hr + start_time_hr
# --- DBSCAN parameters ---
eps = 5
min_samples = 10

# --- Data holders ---
mic_values = []
time_points = [i * 0.5 for i in range(len(image_paths))]  # Every 30 minutes

# === FUNCTIONS ===

def crop_and_process_image(image_path):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Detect circle
    circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, dp=1.2, minDist=100,
                               param1=80, param2=180, minRadius=40, maxRadius=300)
    if circles is not None:
        x, y, r = np.round(circles[0, 0]).astype(int)
        crop = img[y - r:y + r, x - r:x + r]
        return crop, r
    return None, None

def get_dbscan_filtered_points(cropped_img, r):
    hsv = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, np.array([100, 140, 115]), np.array([108, 146, 255]))
    edges = cv2.Canny(mask, 50, 100)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    points = np.vstack(contours).squeeze()

    center = np.array([r, r])
    distances = np.linalg.norm(points - center, axis=1)
    filtered = points[distances < (r - radius_reduction)]

    db = DBSCAN(eps=eps, min_samples=min_samples).fit(filtered)
    labels = db.labels_

    if np.any(labels != -1):
        main_cluster = np.bincount(labels[labels != -1]).argmax()
        return filtered[labels == main_cluster]
    else:
        return filtered

def extract_mic_concentration(points, img_width, direction='right'):
    mid_y = img_width // 2
    tolerance = 5
    xvals = points[np.abs(points[:, 1] - mid_y) < tolerance][:, 0]
    if len(xvals) == 0:
        return np.nan

    if direction == 'right':
        breakthrough_x = np.max(xvals)
        mic = (img_width - breakthrough_x) / img_width * max_conc
    else:
        breakthrough_x = np.min(xvals)
        mic = breakthrough_x / img_width * max_conc
    return mic

# === MAIN LOOP ===

for img_path in image_paths:
    cropped, r = crop_and_process_image(img_path)
    if cropped is None:
        mic_values.append(np.nan)
        continue

    filtered = get_dbscan_filtered_points(cropped, r)
    mic = extract_mic_concentration(filtered, cropped.shape[1], direction=gradient_direction)
    mic_values.append(mic)

# === EXPONENTIAL FIT ===

def exp_decay(t, a, k, c):
    return a * np.exp(-k * t) + c

times = np.array(time_points)
mic_array = np.array(mic_values)
valid_mask = ~np.isnan(mic_array)

params, _ = curve_fit(exp_decay, times[valid_mask], mic_array[valid_mask], p0=[max_conc, 0.1, 0.1])
a, k, c = params
half_life = np.log(2) / k
t_fine = np.linspace(times[0], times[-1], 300)
fitted_curve = exp_decay(t_fine, *params)

# === PLOT ===
plt.figure(figsize=(8, 5))
plt.plot(times, mic_values, 'o', label='Extracted MICs (DBSCAN)', color='darkblue')
plt.plot(t_fine, fitted_curve, '-', label=f'Fitted Decay\n$MIC(t) = {a:.2f}e^{{-{k:.2f}t}} + {c:.2f}$', color='orange')
plt.ax


ValueError: `ydata` must not be empty!